In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def significant(value, s_null, s_alter):
    try:
        if pd.isna(value):
            return value
        elif value < 0.05:
            return s_alter
        else:
            return s_null
    except TypeError:
        return value

In [3]:
test_restuls_path = './significance_test_results/'

In [40]:
def get_significance_test_results(df, s_null, s_alter):
    result_df = pd.DataFrame()
    fold_index = df.columns.get_loc("fold")
    for column in df.columns[fold_index+1:]:
        result_df[column] = df[column].apply(significant, args=(s_null, s_alter))
    return result_df

In [41]:
def get_comparison_results(df_before, df_after, cancer, s_null, s_alter):
    df_comparison = pd.DataFrame(columns=df_before.columns, index=[cancer])
    count = {"improved": 0, "degraded": 0, "no change": 0}
    for column in df_before.columns:
        if column != "Unnamed: 0" and column != "fold":
            for index, row in df_before.iterrows():
                if row[column] == s_alter and df_after.iloc[-1][column] == s_null:
                    df_comparison.loc[cancer, column] = "improved"
                    count["improved"] += 1
                elif row[column] == s_null and df_after.iloc[-1][column] == s_alter:
                    df_comparison.loc[cancer, column] = "degraded"
                    count["degraded"] += 1
                else:
                    df_comparison.loc[cancer, column] = "no change"
                    count["no change"] += 1
    return df_comparison, count

In [49]:
summary_1 = pd.DataFrame()
summary_2 = pd.DataFrame()
for sens_folder in os.listdir(test_restuls_path):
    for cancer_folder in os.listdir(test_restuls_path + sens_folder):
        df1 = pd.read_csv(test_restuls_path + sens_folder + '/' + cancer_folder + '/bias_baseline.csv')
        df2 = pd.read_csv(test_restuls_path + sens_folder + '/' + cancer_folder + '/bias_corrected.csv')
        df3 = pd.read_csv(test_restuls_path + sens_folder + '/' + cancer_folder + '/improvement.csv')
        
        df1_significance = get_significance_test_results(df1, 'fair', 'biased')
        df2_significance = get_significance_test_results(df2, 'fair', 'biased')
        df3_significance = get_significance_test_results(df3, 'no significant', 'significant')
        # print(df1_significance)
        # print(df2_significance)
        df_before = df1_significance.iloc[-1:]
        df_after = df2_significance.iloc[-1:]
        df_improvements = df3_significance.iloc[-2:]

        print('\n\n---Method 1---')
        comparison_results, count = get_comparison_results(df_before, df_after, cancer_folder, 'fair', 'biased')
        print(comparison_results)
        print(cancer_folder)
        print('Improved: ', count['improved'])
        print('Degraded: ', count['degraded'])
        print('No change: ', count['no change'])
        summary_1 = pd.concat([summary_1, comparison_results])

        print('---Method 2---')
        df_improvements.index = [f'{cancer_folder}_0', f'{cancer_folder}_1']
        print(df_improvements)
        summary_2 = pd.concat([summary_2, df_improvements])

    summary_1.to_csv(test_restuls_path + sens_folder + '/summary_1.csv')
    summary_2.to_csv(test_restuls_path + sens_folder + '/summary_2.csv')




---Method 1---
                                                  PQD PQD(class)       EPPV  \
4_lgg_Common Genes_IDH1-Percentage_76.8__2  no change  no change  no change   

                                                 ENPV DPM(Positive)  \
4_lgg_Common Genes_IDH1-Percentage_76.8__2  no change     no change   

                                           DPM(Negative) EOM(Positive)  \
4_lgg_Common Genes_IDH1-Percentage_76.8__2     no change     no change   

                                           EOM(Negative)   AUCRatio  \
4_lgg_Common Genes_IDH1-Percentage_76.8__2     no change  no change   

                                                EOpp0  ...   PPV_diff  \
4_lgg_Common Genes_IDH1-Percentage_76.8__2  no change  ...  no change   

                                             NPV_diff    PR_diff    NR_diff  \
4_lgg_Common Genes_IDH1-Percentage_76.8__2  no change  no change  no change   

                                            BAcc_diff   FPR_diff   FNR_diff  \
4_lg